# Evaluation of different crowd analysis metrics


(1) Dataset filtering:

* Exclude workers based on:
    - contradiction ration (absolut thresh, batch stdv, pair stdv, total stdv)
    - worker quality score (thresholds)
    - attention check fails 
    
(2) Label aggregation:

* Majority vote
* Top vote
* CT unit-label score (tresholds)

In [2]:
from load_data import load_experiment_data, load_gold_data
from evaluation import evaluate_configs

import pandas as pd

run = 4
group = '*'
n_q = '*'
batch = '*'
gold = load_gold_data(run, group, n_q, batch)
print('number of gold instances: ', len(gold))

# load crowd:
run = '*'
group = 'experiment*'
n_q = '*'
batch = '*'
crowd = load_experiment_data(run, group, n_q, batch)

overview_dicts = evaluate_configs(gold, crowd)
df =  pd.DataFrame(overview_dicts) 
df = df.sort_values(by=['relations-f1'], ascending=False)[['config',
                                                      'relations-f1',
                                                      'levels-f1', 
                                                        'negative_relations-f1', 'alpha']]
df.to_csv('../analyses/evaluation_accuracy.csv')
df

number of gold instances:  63
Discarded 655.0 annotations.
----Label distribution----
True: 14
False 49
----------------------------
63 17917 770
aggretation
no filtering - different aggretation methods
cleaning and aggregation


/Users/piasommerauer/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


clean all contradictory annotations


,config,relations-f1,levels-f1,negative_relations-f1,alpha
38,"(batch, 0.5, contradictions, majority_vote)",0.895726,0.930660,0.874621,0.252010
55,"(total, 0.5, contradictions, top_vote)",0.887392,0.861320,0.866315,0.263852
23,"(pair, 0.5, contradictions, top_vote)",0.873016,0.861320,0.849057,0.235974
5,ct_vote_0.65,0.869402,0.925170,0.845138,0.199464
59,"(total, 1, contradictions, top_vote)",0.850693,0.861320,0.823145,0.241196
63,"(total, 1.5, contradictions, top_vote)",0.850693,0.861320,0.823145,0.231196
54,"(total, 0.5, contradictions, majority_vote)",0.850099,0.861320,0.820107,0.263852
58,"(total, 1, contradictions, majority_vote)",0.850099,0.861320,0.820107,0.241196
26,"(pair, 1, contradictions, majority_vote)",0.850099,0.828287,0.820107,0.217212
62,"(total, 1.5, contradictions, majority_vote)",0.850099,0.861320,0.820107,0.231196


In [3]:
#print(df.to_latex(index=False))

# Revision:

Are the wrong answers cases for which we expect disagreement/difficult cases?

Can uqs predict potentially wrong cases? Can contradictions predict potentially wrong cases?

[NOT FINiSHED _ HAS TO BE REVISED]

In [28]:
from utils_analysis import sort_by_key
from load_data import load_experiment_data, load_gold_data
from evaluation import get_evaluation_instances
from clean_annotations import clean_workers
from aggregation import aggregate_binary_labels
from utils_analysis import load_ct
from utils_analysis import load_analysis

from sklearn.metrics import precision_recall_fscore_support as p_r_f1
from collections import defaultdict


def get_correct_incorrect_answers(crowd_eval_agg, gold):
    
    crowd_eval_agg = aggregate_binary_labels(crowd_eval, ct_dicts_units)
    gold_by_triple = sort_by_key(gold, ['relation', 'property', 'concept'])
    corr = dict()
    
    for d in crowd_eval_agg:
        t = f"{d['relation']}-{d['property']}-{d['concept']}"
        gold_d = gold_by_triple[t][0]
        gold_a = str(gold_d['answer']).lower()
        crowd_a = str(d['majority_vote']).lower()
        quid = d['quid']
        if gold_a == crowd_a:
            a = True
        else:
            a = False
        corr[quid] = a
    return corr
            


def detect_low_quality(crowd_eval_agg, ):
    
    predictions =dict()
    worker_cont_rate_thresh = 0.1
    thresh_cont_ratio_pair = 0.5
    uqs_thresh = 0.45

    run = "*"
    group = 'experiment*'
    batch = '*'
    n_q = '*'

    analysis_type = 'workers'
    worker_analysis =  load_analysis(analysis_type, run, group, batch, as_dict=True)
    analysis_by_worker = sort_by_key(worker_analysis, ['workerid'])
    
    analysis_type = 'pairs'
    pair_analysis =  load_analysis(analysis_type, run, group, batch, as_dict=True)
    analysis_by_pair = sort_by_key(pair_analysis, ['pair'])

    analysis_type = 'workers'
    ct_dicts_workers = load_ct(run, group, batch, analysis_type, as_dict=True)
    analysis_type = 'units'
    ct_dicts_units = load_ct(run, group, batch, analysis_type, as_dict=True)
    ct_by_quid = sort_by_key(ct_dicts_units, ['unit'])

    
    quid_workers_dict = defaultdict(list)
    for d in crowd_eval:
        w = d['workerid']
        quid = d['quid']
        quid_workers_dict[quid].append(w)
        
    for d in crowd_eval_agg:
        instance_predictions = []
        pair = f"{d['property']}-{d['concept']}"
        quid = d['quid']
        workers = quid_workers_dict[quid]
        uqs = ct_by_quid[quid][0]['uqs']
        n_contradicting_workers = analysis_by_pair[pair][0]['n_workers_contradicting']
        n_workers  = analysis_by_pair[pair][0]['n_workers']
        cont_ratio = n_contradicting_workers/n_workers
        
        if cont_ratio > thresh_cont_ratio_pair:
            instance_predictions.append(False)
        else:
            instance_predictions.append(True)
        
        worker_cont_rate_above_thresh = 0.0
        for w in workers:
            d = analysis_by_worker[w][0]
            worker_cont_rate = d['contradiction_poss_contradiction_ratio']
            if worker_cont_rate > worker_cont_rate_thresh:
                worker_cont_rate_above_thresh += 1
        if worker_cont_rate_above_thresh/len(workers) > 0.5:
            instance_predictions.append(False)
        else:
            instance_predictions.append(True)
            
        if uqs < uqs_thresh:
            instance_predictions.append(False)
        else:
            instance_predictions.append(True)
        #print(instance_predictions)
        if instance_predictions[2] == False:
            pred  = False
        else:
            pred = True
            
        
        predictions[quid]=pred
    return predictions 

def detect_expected_disagreement():
    pass 

run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

crowd = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
crowd_eval = get_evaluation_instances(crowd, gold)

crowd_eval = clean_workers(crowd_eval, run, group,
                                               batch, 'contradictions', 'total', 1)

analysis_type = 'units'
ct_dicts_units = load_ct(run, group, batch, analysis_type, as_dict=True)
crowd_eval_agg = aggregate_binary_labels(crowd_eval, ct_dicts_units)
run = 4
group = '*'
n_q = '*'
batch = '*'
gold = load_gold_data(run, group, n_q, batch)

corr = get_correct_incorrect_answers(crowd_eval_agg, gold)
predictions = detect_low_quality(crowd_eval_agg)
#predictions = detect_low_quality_pairs(crowd_eval_agg)

correct_incorrect = []
predictions_problematic = []
problematic_cases_identified_correctly = 0.0
for quid, c in corr.items():
    pred = predictions[quid]
    correct_incorrect.append(c)
    if pred == c == False:
        problematic_cases_identified_correctly += 1
    predictions_problematic.append(pred)
    #print(c, pred)
    

print(set(correct_incorrect), set(predictions_problematic))
print('problematic cases identified: ', predictions_problematic.count(False))
print('wrong answers: ', correct_incorrect.count(False) )
print('correctly identified: ', problematic_cases_identified_correctly)
#p, r, f1, support = p_r_f1(correct_incorrect, predictions_problematic, average = 'weighted') 
#print(f'precision {p}, recall {r}, f1 {f1} ')




Discarded 655.0 annotations.
63 17917 770
{False, True} {False, True}
problematic cases identified:  18
wrong answers:  10
correctly identified:  6.0


In [2]:
from load_data import load_experiment_data, load_gold_data
from clean_annotations import clean_workers
from evaluation import evaluate_configs

import pandas as pd

from load_data import load_experiment_data
from calculate_iaa import get_agreement
#from utils_analysis import load_contradiction_pairs
#from utils_analysis import collect_contradictions
from utils_analysis import sort_by_key
#from utils_analysis import get_annotation_ids
from utils_analysis import load_analysis
from utils_analysis import load_ct
from evaluation import get_evaluation_instances
from aggregation import aggregate_binary_labels
#from clean_annotations import remove_contradicting_workers
import statistics

#from collections import Counter, defaultdict
#import pandas as pd
#from scipy.stats import spearmanr

run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

crowd = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
crowd_eval = get_evaluation_instances(crowd, gold)

crowd_eval = clean_workers(crowd_eval, run, group,
                                               batch, 'contradictions', 'total', 1)

run = 4
group = '*'
n_q = '*'
batch = '*'
gold = load_gold_data(run, group, n_q, batch)



Discarded 655.0 annotations.
63 17917 770


In [131]:
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support as p_r_f1

run = "*"
group = 'experiment*'
batch = '*'
n_q = '*'

analysis_type = 'workers'
worker_analysis =  load_analysis(analysis_type, run, group, batch, as_dict=True)

analysis_type = 'units'
ct_dicts_units = load_ct(run, group, batch, analysis_type, as_dict=True)
analysis_type = 'workers'
ct_dicts_workers = load_ct(run, group, batch, analysis_type, as_dict=True)
#ag_cont_dicts = get_agreement_contradiction_data(data_dict_list, pair_analysis, ct_dicts)
#df_ag_cont = pd.DataFrame(ag_cont_dicts)

crowd_eval_agg = aggregate_binary_labels(crowd_eval, ct_dicts_units)

analysis_by_worker = sort_by_key(worker_analysis, ['workerid'])

ct_by_worker = sort_by_key(ct_dicts_workers, ['worker'])
ct_by_unit = sort_by_key(ct_dicts_units, ['unit'])
answers_by_quid = sort_by_key(crowd_eval_agg, ['relation', 'property', 'concept'])

test_run = ['roll', 'red', 'round']
data_by_triple = sort_by_key(crowd_eval, ['relation', 'property', 'concept'])
gold_by_quid = sort_by_key(gold, ['quid'])
gold_by_triple = sort_by_key(gold, ['relation', 'property', 'concept'])

run_dict = defaultdict(list)
answers = []
predictions = []

for triple, data in data_by_triple.items():
    quid = data[0]['quid']
    cont_sum = 0.0
    cont_list = []
    wqs_sum = 0.0
    wqs_list = []
    n_workers = len(data)
    uqs = ct_by_unit[quid][0]['uqs']
    #print(triple, quid)
    answer = str(answers_by_quid[triple][0]['majority_vote']).lower()
    triple = f"{data[0]['relation']}-{data[0]['property']}-{data[0]['concept']}"
    prop = data[0]['property']
    pair = f"{data[0]['property']}-{data[0]['concept']}"
    if prop in test_run:
        test = True
    else:
        test = False
    answer_gold = str(gold_by_triple[triple][0]['answer']).lower()
    iaa = get_agreement(data, v=False, disable_kappa=True)
    if answer == answer_gold:
        correct = True
    else:
        correct = False
    answers.append(correct)
    run_dict[test].append(correct)
    for d in data:
        worker = d['workerid']
        worker_analysis_dict = analysis_by_worker[worker][0]
        cont_rate = worker_analysis_dict['contradiction_poss_contradiction_ratio']
        cont_sum += cont_rate
        cont_list.append(cont_rate)
        wqs_sum += ct_by_worker[worker][0]['wqs']
        wqs_list.append(ct_by_worker[worker][0]['wqs'])
        
    cont_med = statistics.median(cont_list)
    cont_mean = cont_sum / len(cont_list)
    wqs_med = statistics.median(wqs_list)
    
    if  uqs > 0.50: #and correct == True:
        predictions.append(True)
        if correct == True:
            print(triple, wqs_med, answer, answer_gold)
        pass
        #predictions.append(True)
        #print('correct answer identified', triple, correct)
    
        #print('identified:', triple, answer)
    elif uqs <= 0.50: # and correct == False:
        predictions.append(False)
        if correct == False:
            print('incorrect answer identified', triple, correct)
    

        #predictions.append(False)
        #print('risky unit identified:', triple, answer, correct)
  
        #print(triple, correct, cont_med)
p, r, f1, support = p_r_f1(answers, predictions, average = 'weighted') 
print(f'precision {p}, recall {r}, f1 {f1} ')

typical_of_concept-fly-arrow 0.6796334308426818 true true
impossible-fly-arrow 0.6796334308426818 false false
rare-roll-pin 0.6148316539446073 false false
unusual-roll-pin 0.6750428924666586 false false
impossible-roll-pin 0.6750428924666586 false false
incorrect answer identified implied_category-yellow-buttercup False
typical_of_concept-yellow-buttercup 0.7311694232474322 true true
implied_category-roll-tire 0.7551381325387114 true true
typical_of_property-roll-tire 0.7551381325387114 true true
typical_of_concept-roll-tire 0.7551381325387114 true true
afforded_unusual-roll-tire 0.7551381325387114 false false
afforded_usual-roll-tire 0.7551381325387114 true true
variability_limited-roll-tire 0.7551381325387114 false false
creative-roll-tire 0.7551381325387114 false false
rare-roll-tire 0.7551381325387114 false false
impossible-roll-tire 0.7551381325387114 false false
implied_category-roll-shovel 0.6781881476242915 false false
incorrect answer identified afforded_unusual-roll-shovel Fa

7 16
test run False: 0.4375
21 25
test run True: 0.84
